In [ ]:
#Some basic libraries

import numpy as np
import pandas as pd
import gc
import math
import re
import datetime

#Web scraping imports 

import requests
from bs4 import BeautifulSoup as bs
import json
import urllib.request
import time


# For Viz

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

# CREATING ROUTINE FOR EXTRACTION

In [ ]:
# This stays the same 
url_end1 = '&start='

# ADD a += 25 to each loop 
url_end2 = 25

# Feed in number of pages in search
times_run = 0

while times_run in range(5):
    # Read_in the url 
    r = requests.get(f"https://www.linkedin.com/jobs/search/?f_TPR=r86400&geoId=103644278&keywords=data%20analyst%20remote{url_end1}{url_end2}")
    # Convert to a Beautiful Soup Object 
    soup = bs(r.content)
    # Create Variable for just the body
    body = soup.body
        
    # ul class="jobs-search__results-list"
    jobs_container = body.find_all('ul', class_='jobs-search__results-list')
        
    # Creating a list of each li (job listing) within the jobs_container obj
    for x in jobs_container:
        the_li = x.find_all('li')
            
    # THIS grabs the a tags that contain the links to each job  
    job_num_list = []

    for x in the_li:
        job_num = x.find(attrs={"data-id" : ""})
        job_num_list.append(job_num)
            
    # THIS grabs the LINKS themselves and puts them into a list
    link_list = []

    for i in job_num_list:
        linky = i.get('href')
        link_list.append(linky)
            
    # THIS grabs the html for each job posting that we now need 
    for each_link in link_list:
        one_link = requests.get(each_link)
        soup_one_link = bs(one_link.content)
        body_one_link = soup_one_link.body
        # Then we pair down to the h3 tag where the info is 
        pair_down_1 = body_one_link.find_all('h3',class_="topcard__flavor-row")
            
        # Creates a list to hold each jobs info 
        each_job = []

        # Grabs company name
        company_name = body_one_link.find('a', class_="sub-nav-cta__optional-url")
        if company_name:
            each_job.append(company_name.text)

        # Grabs job title 
        job_title = body_one_link.find('h3', class_="sub-nav-cta__header")
        if job_title:
            each_job.append(job_title.text)
            
        # Grabs the number of applicants and time since posted 
        for x in pair_down_1:

            time_ago = x.find('span', class_="topcard__flavor--metadata posted-time-ago__text posted-time-ago__text--new")
            num_applicants = x.find(class_="topcard__flavor--metadata topcard__flavor--bullet num-applicants__caption")
            be_among = x.find(class_='num-applicants__caption')


            if time_ago:
                each_job.append(time_ago.text)
            if num_applicants:
                each_job.append(num_applicants.text)
            if be_among:
                each_job.append(be_among.text)
                  
        master_list_LI.append(each_job)
        time.sleep(2)
        
    times_run += 1
    url_end2 -= 25
    time.sleep(5)

# STRUCTURING DATA

In [ ]:
#Creating full list for date scraped
page_one_april12 = master_list_LI.copy()
the_rest_april12 = master_list_LI.copy()
april12_linked = page_one_april12 + the_rest_april12

# SIMPLE CLEAN FOR SPLIT ENDS 

In [ ]:
#Cleaning 
april12_for_df = [x[:-1] if len(x) > 4 else x for x in april12_linked]

# LOADING TO LOCAL DRIVE FOR FURTHER WORK 

In [ ]:
#Creating DF
april12_DF = pd.DataFrame(april12_for_df, columns = ['company', 'job_title', 'time_elapsed', 'applicants' ])

#Exporting for further work in excel 
april12_DF.to_csv('linked_april_12.csv')